In [1]:
from pyspark.sql.types import DoubleType
sparkDFUser = sqlContext.read.format("csv").options(delimiter="^").load("/FileStore/tables/1u8l0vzz1508009117848/user.csv")
sparkDFReview = sqlContext.read.format("csv").options(delimiter="^").load("/FileStore/tables/1u8l0vzz1508009117848/review.csv")
sparkDFBusiness = sqlContext.read.format("csv").options(delimiter="^").load("/FileStore/tables/1u8l0vzz1508009117848/business.csv")

# Change schema names
prevColBus = sparkDFBusiness.schema.names
newColBus = ["business_id", "full_address", "categories"]
dfBus = reduce(lambda data, idx: data.withColumnRenamed(prevColBus[idx], newColBus[idx]), xrange(len(prevColBus)), sparkDFBusiness)

prevColRev = sparkDFReview.schema.names
newColRev = ["review_id", "user_id", "business_id", "stars"]
dfRev = reduce(lambda data, idx: data.withColumnRenamed(prevColRev[idx], newColRev[idx]), xrange(len(prevColRev)), sparkDFReview)

prevColUser = sparkDFUser.schema.names
newColUser = ["user_id", "name", "url"]
dfUser = reduce(lambda data, idx: data.withColumnRenamed(prevColUser[idx], newColUser[idx]), xrange(len(prevColUser)), sparkDFUser)

# Convert to RDD
rUsers = sparkDFUser.rdd
rReview = sparkDFReview.rdd
rBusiness = sparkDFBusiness.rdd

In [2]:
rReview.take(5)
rBusiness.map(lambda x : (x[0], (x[1], x[2]))).take(5)

In [3]:
# Q1

# Drop unecessary columns, format into (K, V) pairs
rddRevs = rReview.map(lambda x : (x[2], float(x[3])))
rddBusi = rBusiness.map(lambda x : (x[0], (x[1], x[2])))

# Get average star ratings for each business
rddRevAvg = rddRevs.aggregateByKey((0, 0), lambda a,b: (a[0] + b,    a[1] + 1), lambda a,b: (a[0] + b[0], a[1] + b[1])).mapValues(lambda x: x[0]/x[1])

# Join on business_id and flatten
rRB = rddRevAvg.join(rddBusi).map(lambda x : (x[0], x[1][1][0], x[1][1][1], x[1][0]))

# Take top 10 by rating
q1Out = rRB.takeOrdered(10, lambda x : -x[3])
q1Out

In [4]:
# Q2
# *************** SPECIFY USER *************** 
user = "Shannon L."

# Reformat
rddQ2Users = rUsers.map(lambda x : (x[0], (x[1], x[2])))
rddQ2Revs = rReview.map(lambda x : (x[1], (x[0], x[2], x[3])))

# Join on user_id
rddUseRevs = rddQ2Users.join(rddQ2Revs)

# Flatten and drop unecessary columns
rddUserRatings = rddUseRevs.map(lambda x : (x[1][0][0], float(x[1][1][2])))

# Reduce by specified user name and find their average rating
rddUserRatings = rddUserRatings.filter(lambda x : x[0] == user).aggregateByKey((0, 0), lambda a,b: (a[0] + b,    a[1] + 1), lambda a,b: (a[0] + b[0], a[1] + b[1])).mapValues(lambda x: x[0]/x[1])

# Output
rddUserRatings.take(1)

In [5]:
#Q3

# Filter for only businesses with 'Stanford' in their address
rddStanford = rBusiness.filter(lambda x : "Stanford" in x[1]).map(lambda x : (x[0], 0))

# Remove unecessary columns 
rddQ3Rev = rReview.map(lambda x : (x[2], (x[1], float(x[3]))))

# Join and flatten on business_id
q3Out = rddStanford.join(rddQ3Rev).map(lambda x : (x[1][1][0], x[1][1][1]))
q3Out.take(10)

In [6]:
# Q4
rReview.map(lambda x : (x[1])).map(lambda x : (x, 1)).reduceByKey(lambda x, y : x + y).join(rUsers).map(lambda x : (x[0], x[1][0], x[1][1])).sortBy(lambda x : x[1], ascending=False).map(lambda x : (x[0], x[2])).take(10)

In [7]:
#Q5
rBusiness.filter(lambda x : 'Texas' in x[1]).map(lambda x : (x[0], 0)).join(rReview.map(lambda x : (x[2], (x[0], x[1], x[3])))).map(lambda x : (x[0], 1)).reduceByKey(lambda x, y : x + y).collect()